# Проект по "Введению в финансовые рынки" по теме "Дебиторская задолженность"

### **0. Установка зависимостей**

Зависимости и их версии перечислены в файле `requirements.txt`

In [ ]:
!pip3 install requirements.txt

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns
import optuna

from catboost import CatBoostClassifier, Pool

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, mean_absolute_error, r2_score, roc_curve, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor


### **1. Ознакомление с данными и их предварительная обработка**

In [ ]:
dataset_path = './data/ar_dataset.csv'
data = pd.read_csv(dataset_path)
data.head(10)

In [ ]:
data.info()

### Описание полей датасета

+ `countryCode`: Код страны клиента.
+ `customerID`: Уникальный идентификатор клиента.
+ `PaperlessDate`: Дата перехода на электронные счета.
+ `invoiceNumber`: Номер счета.
+ `InvoiceDate`: Дата создания счета.
+ `DueDate`: Дата, к которой нужно было оплатить счет.
+ `InvoiceAmount`: Сумма счета.
+ `Disputed`: Указывает, был ли счет оспорен.
+ `SettledDate`: Дата оплаты счета.
+ `PaperlessBill`: Тип счета (бумажный / электронный).
+ `DaysToSettle`: Количество дней, которое потребовалось для оплаты счета.
+ `DaysLate`: Количество дней просрочки оплаты.

Преобразовываем столбцы с датой и временем в нужный нам формат

In [ ]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
data['DueDate'] = pd.to_datetime(data['DueDate'])
data['SettledDate'] = pd.to_datetime(data['SettledDate'])

Вычисляем DaysOverdue для проверки корректности DaysLate, учета пропусков и создания более гибких таргетов для анализа и моделей

In [ ]:
data['DaysOverdue'] = (data['SettledDate'] - data['DueDate']).dt.days.clip(lower=0)

Сравниваем

In [ ]:
data.loc[data['DaysOverdue'] != data['DaysLate'], ['DaysOverdue', 'DaysLate']]

Так-как `DaysOverdue` и `DaysLate` корректны - продолжаем работу с `DaysLate`.

У нас есть таргет для регрессии DaysLate, но ещё нужен бинарный таргет для классификации того, была ли компанией просрочена оплата ранее или нет. Таковым будет являться `OnTimePayment`.

In [ ]:
data['OnTimePayment'] = (data['DaysLate'] > 0).astype(int)
data[['customerID','DaysLate','OnTimePayment']]

Посмотрим на корреляцию между суммой счета, количеством дней оплаты счета и просрочкой оплаты

In [ ]:
correlation = data.loc[:, ~data.columns.isin(['OnTimePayment', 'DaysOverdue'])].corr(numeric_only=True)
plt.figure(figsize=(10, 6))
sns.heatmap(correlation, annot=True, cmap='coolwarm', fmt='.2f')
plt.title("Корреляционная матрица")
plt.show()

По тепловой карте можно понять, что:
 + `DaysToSettle` имеет высокую положительную корреляцию с `DaysLate` (0.82).
     * Это ожидаемо, так как количество дней урегулирования связано с задержками.(т.е. нам не нужен этот признак для обучения модели)
 + `InvoiceAmount` имеет слабую положительную корреляцию с `DaysLate` (0.06)
     * Это может указывать на слабое влияние размера счета на просрочку.
 + `countryCode` тоже имеет положительную корреляцию с `DaysLate` (0.09)
     * Это может указывать на некоторое влияние страны компании на просрочку.
 + `InvoiceNumber` никак не коррелирует с `DaysLate` (0.00)

Так как нам нужно найти такие признаки которые нам будут известны ещё до предоставления задолженности, то мы из данных признаков учтем только `InvoiceAmount` и `countryCode` и продолжим анализ.

***Изначально доступные признаки:***
+ `countryCode`
+ `InvoiceAmount`
+ `PaperlessBill`
+ `Disputed`

***Поздние признаки (целевые переменные):***

+ `DaysToSettle`
+ `DaysLate`
+ `LatePayment`

## 1.1 Исследование числовых признаков

Нужно анализировать распределение сумм счетов и понимать возможные выбросы, чтобы избежать от переобучения модели в будущем

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(data['InvoiceAmount'], bins=30, kde=True)
plt.title("Распределение суммы счетов (InvoiceAmount)")
plt.xlabel("InvoiceAmount")
plt.ylabel("Частота")
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(data['InvoiceAmount'])
plt.title("Boxplot суммы счетов (InvoiceAmount)")
plt.show()

**Вывод:** Суммы счетов распределены в основном равномерно, но есть маленькое количество выбросов которые можно обработать логарифмируя значения сумм. 

Теперь рассмотрим распределение просрочек платежа

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(data['DaysLate'], bins=30, kde=True)
plt.title("Распределение количества дней просрочки (DaysLate)")
plt.xlabel("DaysLate")
plt.ylabel("Частота")
plt.show()

**Вывод:** Большинство значений близко к 0, что подтверждает, что большинство платежей осуществляется вовремя. Есть длинный хвост для значительных задержек (10+ дней).

### **1.1. Исследование влияния категориальных признаков на просрочку**

Влияние `PaperlessBill` на `DaysLate`

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(x='PaperlessBill', y='DaysLate', data=data)
plt.title("PaperlessBill vs DaysLate")
plt.xlabel("Тип счета (PaperlessBill)")
plt.ylabel("Количество дней просрочки (DaysLate)")
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data=data, x='DaysLate', hue='PaperlessBill', bins=20, kde=True, alpha=0.6)
plt.title("Гистограмма просрочки (DaysLate) по PaperlessBill")
plt.xlabel("DaysLate")
plt.ylabel("Количество")
plt.show()

**Вывод:** Электронные счета `Electronic` имеют меньшую вероятность задержки. Основная масса счетов имеет `DaysLate` близким к нулю.
Бумажные счета `Paper` демонстрируют большее количество задержек по сравнению с электронными счетами, что заметно по более длинному хвосту в распределении.

Влияние `Disputed` на `DaysLate`

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(x='Disputed', y='DaysLate', data=data)
plt.title("PaperlessBill vs DaysLate BoxPlot")
plt.xlabel("Тип счета (PaperlessBill)")
plt.ylabel("Количество дней просрочки (DaysLate)")
plt.show()

plt.figure(figsize=(10, 6))
sns.histplot(data=data, x='DaysLate', hue='Disputed', bins=20, kde=True, alpha=0.6)
plt.title("Disputed vs DaysLate Histogramm")
plt.xlabel("DaysLate")
plt.ylabel("Количество")
plt.show()

**Вывод:** Оспоренные счета имеют больше вероятность просрочки платежа, чем неоспоренные

### **1.2. Коррекция датасета для обучения модели**

Прологорифмуем все `InvoiceAmount` для обработки выбросов

In [ ]:
data['LogInvoiceAmount'] = np.log1p(data['InvoiceAmount'])

plt.figure(figsize=(10, 6))
sns.histplot(data['LogInvoiceAmount'], bins=20, kde=True, color='orange', alpha=0.7)
plt.title("Гистограмма логарифма суммы счетов (LogInvoiceAmount)")
plt.xlabel("LogInvoiceAmount")
plt.ylabel("Частота")
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
sns.boxplot(data['LogInvoiceAmount'])
plt.title("Boxplot суммы логарифмов счетов (LogInvoiceAmount)")
plt.show()

После логарифмирования нет слишком больших счетов, которые бы могли сильно повлиять на обучения модели.

Далее превратим категориальные признаки в числовые используя `one-hot encoding`, чтобы модель сумела на их основе обучиться.

In [ ]:
data_encoded = pd.get_dummies(data[['countryCode', 'LogInvoiceAmount','DaysLate', 'OnTimePayment','PaperlessBill','Disputed']], drop_first=True)
data_encoded

## **2. Обучение модели**

Разделение на обучающие и тестовые выборки

In [ ]:
X = data_encoded.drop(columns=['DaysLate', 'OnTimePayment'])
y = data_encoded['OnTimePayment'] 
y_reg = data_encoded[['DaysLate']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y_reg, test_size=0.2, random_state=42)

Для данной задачин было решено использовать 3 алгоритма и сравнить результаты.

## **2.1. KNN**
Для данной задачи классификации было решено использовать алгоритм KNN (K-Nearest-Neighbours), так-как это простой, но мощный алгоритм, который может хорошо работать с небольшими или умеренными наборами данных. Он классифицирует объект на основе метрики близости к соседним объектам. 

#### Нормализация данных
Нормализация выравнивает масштабы данных, что важно для KNN, так как он основан на расстояниях.

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train_scaled, y_train)

y_pred = knn_model.predict(X_test_scaled)

Оценка качества

In [ ]:
accuracy_knn = accuracy_score(y_test, y_pred)
print(f"Точность модели KNN: {accuracy_knn:.2f}")

report = classification_report(y_test, y_pred)
print("Отчет классификации:")
print(report)

Для улучшения модели попытаемся выбрать оптимальное количество соседей

In [ ]:
param_grid = {'n_neighbors': range(1, 50)}
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_scaled, y_train)

best_k = grid_search.best_params_['n_neighbors']
print(f"Лучшее значение k: {best_k}")

best_knn_accuracy = grid_search.best_score_
print(f"Точность с оптимальным k: {best_knn_accuracy:.2f}")

**Вывод:** С помощью алгоритма KNN мы смогли достичь точности в 73%.

## **2.2. RandomForest**

Как 2-й подход для решения данной задачи был выбран RandomForest, так-как этот подход для классификации или регрессии является хорошим выбором в ряде случаев из-за его характеристик.
С помощью нее мы не только решили не только предугадать будет ли задолженность просрочена или нет, но еще и предсказать на сколько дней.

### **2.2.1. RandomForest Classifier**

In [ ]:
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train)
y_pred_reg = rf_classifier.predict(X_test)

accuracy_rf_class = accuracy_score(y_test, y_pred)
print(f"Точность модели: {accuracy_rf_class:.2f}")

report = classification_report(y_test, y_pred)
print("Отчет классификации:")
print(report)

In [ ]:
feature_importances = pd.Series(rf_classifier.feature_importances_, index=X.columns)
feature_importances.sort_values(ascending=False, inplace=True)

# Визуализация важности признаков
plt.figure(figsize=(10, 6))
feature_importances.plot(kind='bar')
plt.title("Важность признаков")
plt.ylabel("Важность")
plt.show()

**Вывод:** 
+ Общий обзор предсказаний
    * 74% объектов, предсказанных как "оплачено вовремя", действительно принадлежат этому классу.
    * 83% всех объектов класса "оплачено вовремя" модель правильно классифицировала.
    * 60% предсказанных "с просрочкой" действительно имеют задержку.
    * Модель нашла только 47% всех объектов с задержкой.

+ Важность признаков в `Random Forest Regressor`
  - Сумма счета - самый важный признак с важностью около 0.65
  - Оспоренные счета также значительно влияют на задержки платежей ~ 0.2

### **2.2.2. RandomForest Regressor**

In [ ]:
rf_regressor = RandomForestRegressor(random_state=42, n_estimators=100)
rf_regressor.fit(X_train_reg, y_train_reg)
y_pred = rf_regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)  # Среднеквадратичная ошибка
mae = mean_absolute_error(y_test, y_pred)  # Средняя абсолютная ошибка
r2 = r2_score(y_test, y_pred)  # Коэффициент детерминации R^2

# Вывод метрик
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"R^2 (коэффициент детерминации): {r2:.2f}")

In [ ]:
feature_importances = pd.Series(rf_regressor.feature_importances_, index=X.columns)
feature_importances.sort_values(ascending=False, inplace=True)

plt.figure(figsize=(5, 4))
feature_importances.plot(kind='bar')
plt.title("Важность признаков в Random Forest Regressor")
plt.ylabel("Важность")
plt.show()


**Вывод:**
+ Метрики модели `Random Forest Regressor`:
  * Среднеквадратичная ошибка `MSE`: 26.99

    - Это означает, что в среднем квадрат отклонения предсказанных значений `DaysLate` от фактических составляет около 27.
    - `MSE` чувствительна к выбросам: большие ошибки сильнее влияют на итоговую метрику.
  * Средняя абсолютная ошибка `MAE`: 3.14

    - Модель в среднем ошибается примерно на 3.14 дня, что является более интерпретируемой метрикой, чем `MSE`.
    - Это значение указывает на относительное качество модели.
  * Коэффициент детерминации `R²`: -117.59

    - Отрицательное значение `R²` говорит о том, что модель работает хуже, чем простое предсказание среднего значения. Это указывает на проблемы в обучении модели или неподходящие данные.

+ Важность признаков в `Random Forest Regressor`
  - Сумма счета - самый важный признак с важностью около 0.65
  - Оспоренные счета также значительно влияют на задержки платежей ~0.2

### Сравнение точности  2-х моделей классификации

In [ ]:
models = ['Random Forest', 'KNN']
accuracies = [accuracy_rf_class, best_knn_accuracy]  

plt.figure(figsize=(5, 5))
plt.bar(models, accuracies, color=['blue', 'orange', 'green'])
plt.title('Сравнение Accuracy между моделями')
plt.ylabel('Accuracy')
plt.ylim(0, 1)
plt.show()

###  ROC-кривая для сравнения моделей классификации
ROC-кривая показывает, насколько хорошо модель различает классы. Она строится на основе метрик True Positive Rate (TPR) и False Positive Rate (FPR).


In [ ]:
plt.figure(figsize=(8, 6))

# Random Forest
fpr_rf, tpr_rf, _ = roc_curve(y_test, rf_classifier.predict_proba(X_test)[:, 1])
auc_rf = roc_auc_score(y_test, rf_classifier.predict_proba(X_test)[:, 1])
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (AUC = {auc_rf:.2f})', color='blue')

# KNN
fpr_knn, tpr_knn, _ = roc_curve(y_test, knn_model.predict_proba(X_test_scaled)[:, 1])
auc_knn = roc_auc_score(y_test, knn_model.predict_proba(X_test_scaled)[:, 1])
plt.plot(fpr_knn, tpr_knn, label=f'KNN (AUC = {auc_knn:.2f})', color='orange')

# линия слу
plt.plot([0, 1], [0, 1], 'k--', label='Random')

plt.title('Сравнение ROC-кривых между моделями')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.grid()
plt.show()

**Вывод:** На основе ROC-кривых видно, что модели (Random Forest и KNN) имеют схожую производительность: AUC для KNN составляет 0.69, а для Random Forest — 0.68. Хотя KNN показывает немного лучшее качество, разница минимальна, и обе модели работают лишь чуть лучше случайного предсказания. 

## **2.3. Binary Classification**

In [ ]:
df = pd.read_csv('./data/ar_dataset.csv')
df

In [ ]:
duplicate_counts = df['countryCode'].value_counts() - 1
duplicate_counts = len(duplicate_counts > 0)
duplicate_counts

In [ ]:
df_classification = df.copy()

In [ ]:
df_classification['DaysLate'] = df_classification['DaysLate'].apply(lambda x: 1 if x > 0 else x)

In [ ]:
df_classification = df_classification.drop(columns=['DaysToSettle'])

In [ ]:
df_classification

In [ ]:
def split_date_column(df, date_column):
    """
    Splits a date column into day, month, and year columns, drops the original column,
    and ensures the order of columns remains the same.

    Args:
        df (pd.DataFrame): Input DataFrame.
        date_column (str): Name of the column containing date strings.

    Returns:
        pd.DataFrame: DataFrame with new columns for day, month, and year, in the correct order.
    """
    # Ensure the column is in datetime format
    df[date_column] = pd.to_datetime(df[date_column], format='%m/%d/%Y')

    # Extract day, month, and year into new columns
    base_name = date_column[:-4]  # Strip "Date" suffix
    new_columns = {
        f'{base_name}Day': df[date_column].dt.day,
        f'{base_name}Month': df[date_column].dt.month,
        f'{base_name}Year': df[date_column].dt.year
    }

    # Get the original column order
    original_columns = df.columns.tolist()

    # Find the index of the date column
    date_col_index = original_columns.index(date_column)

    # Drop the original date column
    df.drop(columns=[date_column], inplace=True)

    # Insert new columns in the correct order
    for i, (col_name, col_data) in enumerate(new_columns.items()):
        df.insert(date_col_index + i, col_name, col_data)

    return df

In [ ]:
# Converting a date format column into 3 separate columns for day, month, and year

df_classification = split_date_column(df_classification, 'PaperlessDate')
df_classification = split_date_column(df_classification, 'InvoiceDate')
df_classification = split_date_column(df_classification, 'SettledDate')
df_classification = split_date_column(df_classification, 'DueDate')

In [ ]:
df_classification

In [ ]:
df_classification['Disputed'] = df_classification['Disputed'].apply(lambda x: 1 if x == 'Yes' else 0)


In [ ]:
df_classification.dtypes

In [ ]:
columns_to_change = ['countryCode', 'customerID', 'PaperlessBill', 'Disputed']
df_classification[columns_to_change] = df_classification[columns_to_change].astype('category')

In [ ]:
df_classification.dtypes

In [ ]:
# Splitting the data into train and test sets
def prepare_data(df, target, cat_features, test_size=0.5, random_state=42):
    X = df.drop(columns=[target])
    y = df[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test, cat_features

# Objective function for Optuna
def objective(trial, X_train, X_test, y_train, y_test, cat_features):
    params = {
        "iterations": trial.suggest_int("iterations", 100, 1000),
        "depth": trial.suggest_int("depth", 4, 12),
        "learning_rate": trial.suggest_float("learning_rate", 1e-4, 0.5, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 10),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0, 1),
        "border_count": trial.suggest_int("border_count", 1, 255),
        "random_strength": trial.suggest_float("random_strength", 1e-4, 10, log=True),
        "verbose": 0,
        "eval_metric": "AUC",
        "task_type": "CPU",  # Change to 'GPU' if you have GPU support
    }
    
    # Create CatBoost Pool for train and test
    train_pool = Pool(X_train, y_train, cat_features=cat_features)
    test_pool = Pool(X_test, y_test, cat_features=cat_features)

    # Train CatBoost
    model = CatBoostClassifier(**params)
    model.fit(train_pool, eval_set=test_pool, early_stopping_rounds=50, verbose=False)
    
    # Evaluate ROC AUC score
    y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probability of class 1
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    return roc_auc

# Main function for training and hyperparameter tuning
def tune_and_train(df, target, cat_features, n_trials=50):
    X_train, X_test, y_train, y_test, cat_features = prepare_data(df, target, cat_features)
    

    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective(trial, X_train, X_test, y_train, y_test, cat_features), n_trials=n_trials)

    print("Best parameters:", study.best_params)
    print("Best ROC AUC score:", study.best_value)
    
    # Train final model with the best parameters
    best_params = study.best_params
    final_model = CatBoostClassifier(**best_params)
    final_model.fit(Pool(X_train, y_train, cat_features=cat_features), verbose=False)
    
    # Evaluate the final model
    y_pred_proba = final_model.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    print("Final model ROC AUC on test set:", roc_auc)
    return final_model, study.best_params

# Example usage
cat_features = ['countryCode', 'customerID', 'PaperlessBill', 'Disputed']

final_model, best_params = tune_and_train(df_classification, target="DaysLate", cat_features=cat_features)


In [ ]:
print("Best parameters:", best_params)

In [ ]:
a = final_model.predict(df_classification.drop(columns=["DaysLate"]))

In [ ]:
b = df_classification["DaysLate"].to_list()

In [ ]:
max([a_i - b_i for a_i, b_i in zip(a, b)])